## Description

Notebook holds code for:
1. generating ip ranges
2. creating connection between jdbc db(write and read)
3. solution for the problem 

### Generator for ip ranges 
Generator is builed with python and creates ranges that can have length between 1 and 100. At the end csv file with data is created.

In [1]:
%%python
import random
import socket
import struct
socket.inet_ntoa(struct.pack('>I', random.randint(1, 0xffffffff)))
import csv  
fields = ['id', 'start', 'end']  
filename = "ips.csv"

import socket
import struct


def ip2int(addr):
    return struct.unpack("!I", socket.inet_aton(addr))[0]


def int2ip(addr):
    return socket.inet_ntoa(struct.pack("!I", addr))

def generate_ips_csv(number_of_ranges):
    with open(filename, 'w') as csvfile:  
        csvwriter = csv.writer(csvfile)  

        csvwriter.writerow(fields)  

        for i in range(number_of_ranges):
            startIp = socket.inet_ntoa(struct.pack('>I', random.randint(1, 0xffffffff)))

            randomRange = random.randint(1, 100)

            endIp = int2ip(ip2int(startIp)+randomRange)
            row = [i, startIp, endIp]
            csvwriter.writerow(row) 


In [2]:
%%python
generate_ips_csv(10)

### Scala Init

In [3]:
val x:Int = 0
import org.apache.spark.sql._

Intitializing Scala interpreter ...

Spark Web UI available at http://3c68ebd29f52:4041
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1615218795351)
SparkSession available as 'spark'


x: Int = 0
import org.apache.spark.sql._


### Prepare data


In [4]:
def prepareIntervalIpInLongDf(df: DataFrame, startColumnName: String, endColumnName: String): DataFrame = {
     df
    .withColumn(startColumnName, udfIpToLong(col(startColumnName)))
    .withColumn(endColumnName, udfIpToLong(col(endColumnName)))
}


def ipToLong(ipAddress: String): Long = {
   ipAddress.split("\\.").reverse.zipWithIndex.map(a=>a._1.toInt*math.pow(256,a._2).toLong).sum
}

def longToIP(long: Long): String = {
   (0 until 4).map(a=>long / math.pow(256, a).floor.toInt % 256).reverse.mkString(".")
}

val udfIpToLong = udf((s: String) => Option(s).map(str => ipToLong(str)))
val udfLongToIp = udf((s: Long) => Option(s).map(ipLong => longToIP(ipLong)))


prepareIntervalIpInLongDf: (df: org.apache.spark.sql.DataFrame, startColumnName: String, endColumnName: String)org.apache.spark.sql.DataFrame
ipToLong: (ipAddress: String)Long
longToIP: (long: Long)String
udfIpToLong: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$2066/0x0000000840c27040@2013f6b7,LongType,List(Some(class[value[0]: string])),None,true,true)
udfLongToIp: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$2108/0x0000000840cd6040@202a4b9c,StringType,List(Some(class[value[0]: bigint])),None,true,true)


### Create set of up all ip addresses from ranges and delete those that are in other ranges

In [5]:
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.lag
import org.apache.spark.sql.expressions.WindowSpec 

def range(start: Long, end: Long): List[Long] = (start to end).toList

val udfLongIntervalToRange = udf((start: Long, end: Long) => range(start, end))

def getMutuallyExclusiveIps(ipDf: DataFrame, identityColumnName: String, startIpColumnName: String, endIpColumnName: String): DataFrame = {
        
    ipDf
    .withColumn("ipAddreses", udfLongIntervalToRange(col(startIpColumnName),col(endIpColumnName)))
    .withColumn(identityColumnName,explode_outer($"ipAddreses"))
    .select(identityColumnName)
    .groupBy(col(identityColumnName))
    .count
    .where($"count"===1)
}


def createIntervalsFromContinuousNumbers(mutuallyExclusiveDf: DataFrame, identityColumnName: String, startColumnName: String, endColumnName: String): DataFrame = {
    
    
    val windowSpec  = Window.orderBy(identityColumnName)
    val windowSpecDesc  = Window.orderBy(desc(identityColumnName))

    val startIpInterval = identifyContinousNumbers(mutuallyExclusiveDf, windowSpec, identityColumnName, startColumnName)

    val endIpInterval = identifyContinousNumbers(mutuallyExclusiveDf, windowSpecDesc, identityColumnName, endColumnName)

    startIpInterval.join(endIpInterval, Seq("row_no")).select(startColumnName, endColumnName)
}


def identifyContinousNumbers(
    mutuallyExclusiveDf: DataFrame, 
    windowSpec: WindowSpec,
    identityColumnName: String, 
    intervalColumnName: String):DataFrame = {
    

    mutuallyExclusiveDf
    .withColumn("contiguous_grp", col(identityColumnName) - lag(identityColumnName, 1,0).over(windowSpec))
    .where(abs($"contiguous_grp")=!=1)
    .groupBy("contiguous_grp")
    .agg(first(identityColumnName).alias(identityColumnName))
    .withColumn(intervalColumnName, udfLongToIp(col(identityColumnName)))
    .withColumn("row_no", row_number().over(Window.orderBy(intervalColumnName)))
    .drop("contiguous_grp")
}


import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.lag
import org.apache.spark.sql.expressions.WindowSpec
range: (start: Long, end: Long)List[Long]
udfLongIntervalToRange: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$2141/0x0000000840cec040@8fc1fd3,ArrayType(LongType,false),List(Some(class[value[0]: bigint]), Some(class[value[0]: bigint])),None,true,true)
getMutuallyExclusiveIps: (ipDf: org.apache.spark.sql.DataFrame, identityColumnName: String, startIpColumnName: String, endIpColumnName: String)org.apache.spark.sql.DataFrame
createIntervalsFromContinuousNumbers: (mutuallyExclusiveDf: org.apache.spark.sql.DataFrame, identityColumnName: String, startColumnName: String, endColumnName: String)org.apache.spark.sql....


In [6]:
def findMutuallyExclusiveIpIntervals(df: DataFrame, startColumnName: String, endColumnName: String): DataFrame = {
    
    val identityColumn = "ipLong"
    
    val prepareDf = prepareIntervalIpInLongDf(df, startColumnName, endColumnName)
    
    val mutuallyExlusiveIp = getMutuallyExclusiveIps(prepareDf, identityColumn, startColumnName, endColumnName)
    
    createIntervalsFromContinuousNumbers(mutuallyExlusiveIp, identityColumn, startColumnName, endColumnName)
}

findMutuallyExclusiveIpIntervals: (df: org.apache.spark.sql.DataFrame, startColumnName: String, endColumnName: String)org.apache.spark.sql.DataFrame


## FIRST TEST CASE

In [7]:
val startColumnName = "ip_start"
val endColumnName = "ip_end"

val df = Seq(("197.203.0.0", "197.206.9.255"),
             ("197.204.0.0", "197.204.0.24"),
             ("201.233.7.160", "201.233.7.168"),
             ("201.233.7.164", "201.233.7.168"),
             ("201.233.7.167", "201.233.7.167"),
             ("203.133.1.0", "203.133.1.16")).toDF(startColumnName, endColumnName)

findMutuallyExclusiveIpIntervals(df, startColumnName, endColumnName).show()

+-------------+---------------+
|     ip_start|         ip_end|
+-------------+---------------+
|  197.203.0.0|197.203.255.255|
| 197.204.0.25|  197.206.9.255|
|201.233.7.160|  201.233.7.163|
|  203.133.1.0|   203.133.1.16|
+-------------+---------------+



startColumnName: String = ip_start
endColumnName: String = ip_end
df: org.apache.spark.sql.DataFrame = [ip_start: string, ip_end: string]


# Whole flow at once


In [8]:
%%python
generate_ips_csv(5)

### Set of properties for db connection

In [9]:
import java.util.Properties

val fileNameWithData = "ips.csv"
val url = "jdbc:mysql://mysql/test"
val driver = "com.mysql.jdbc.Driver"
val dbtable = "test"
val user = "user"
val password = "password"

val props = new Properties()
props.setProperty("user", user)
props.setProperty("password", password)
props.setProperty("driver", driver)


import java.util.Properties
fileNameWithData: String = ips.csv
url: String = jdbc:mysql://mysql/test
driver: String = com.mysql.jdbc.Driver
dbtable: String = test
user: String = user
password: String = password
props: java.util.Properties = {password=password, driver=com.mysql.jdbc.Driver, user=user}
res1: Object = null


### Test jdbc connection. Read generated data from .csv file and save to db

In [10]:
val ipAddresses = spark.read.format("csv").option("header", "true").load("ips.csv").toDF("id","start","end")
ipAddresses.write.mode(SaveMode.Overwrite).option("truncate", true).jdbc(url,dbtable, props)

ipAddresses: org.apache.spark.sql.DataFrame = [id: string, start: string ... 1 more field]


### Read data from db and find mutually exclusive ip intervals

In [11]:
val ipDataFrame = spark.read.jdbc(url,dbtable, props)
ipDataFrame.show
findMutuallyExclusiveIpIntervals(ipDataFrame, "start", "end").show


+---+---------------+---------------+
| id|          start|            end|
+---+---------------+---------------+
|  0|  70.62.141.155|  70.62.141.156|
|  1|   101.149.4.71|   101.149.4.91|
|  2|  248.85.30.228|   248.85.31.11|
|  3|  207.45.130.89| 207.45.130.134|
|  4|205.234.224.135|205.234.224.229|
+---+---------------+---------------+

+---------------+---------------+
|          start|            end|
+---------------+---------------+
|   101.149.4.71|   101.149.4.91|
|205.234.224.135|205.234.224.229|
|  207.45.130.89| 207.45.130.134|
|  248.85.30.228|   248.85.31.11|
|  70.62.141.155|  70.62.141.156|
+---------------+---------------+



ipDataFrame: org.apache.spark.sql.DataFrame = [id: string, start: string ... 1 more field]
